# PYTORCH CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/torch_cnn_classifier.ipynb torch_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
# DATASET = 'no_dataset'
DATASET = 'demo_human_or_worm'
VERSION = 0
BATCH_SIZE = 32
EPOCHS = 1

In [2]:
# Parameters
DATASET = "demo_human_or_worm"
EPOCHS = 10


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_human_or_worm 0 32 10


## Config

In [4]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from genomic_benchmarks.models.torch import CNN
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config, VARIABLE_LENGTH_DATASETS

In [5]:
USE_PADDING = DATASET in VARIABLE_LENGTH_DATASETS
    
config = {
    "dataset": DATASET,
    "dataset_version": VERSION,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "use_padding": USE_PADDING,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 9
{'<eos>': 6, 'G': 5, 'C': 4, 'A': 3, '<pad>': 8, 'T': 2, '<bos>': 1, 'N': 7, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device


max_seq_len  200


not all sequences are of the same length


## Model

In [9]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [10]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/home/jupyter/genomic_benchmarks/src/genomic_benchmarks/dataset_getters/utils.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 90.4%, Avg loss: 0.546975 

Epoch 1


Train metrics: 
 Accuracy: 92.3%, Avg loss: 0.540682 

Epoch 2


Train metrics: 
 Accuracy: 92.4%, Avg loss: 0.538900 

Epoch 3


...

Train metrics: 
 Accuracy: 93.0%, Avg loss: 0.535593 

Epoch 6


Train metrics: 
 Accuracy: 93.3%, Avg loss: 0.534897 

Epoch 7


Train metrics: 
 Accuracy: 93.0%, Avg loss: 0.535228 

Epoch 8


Train metrics: 
 Accuracy: 93.2%, Avg loss: 0.534641 

Epoch 9


Train metrics: 
 Accuracy: 92.9%, Avg loss: 0.535920 



## Testing

In [11]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  11321.268815517426 ; fp  654.7251256960077
recall  0.905701505241394 ; precision  0.9453302056672827
num_batches 782
correct 23172
size 25000
Test metrics: 
 Accuracy: 0.926880, F1 score: 0.925092, Avg loss: 0.536891 



(0.92688, 0.9250916504317582)